In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
!pip install fairlearn
!pip install lime
from fairlearn.metrics import (
    MetricFrame,
    selection_rate,
    true_positive_rate
)


# Step 1: Load dataset Student Performance
df = pd.read_csv("/content/StudentsPerformance.csv")


# Step 2: Basic preprocessing
# Encode categorical features
label_enc = LabelEncoder()
for col in df.select_dtypes(include="object").columns:
    df[col] = label_enc.fit_transform(df[col])

# Create target variable: 1 if passed math (>50), else 0
df['passed_math'] = (df['math score'] >= 50).astype(int)

# Sensitive attribute (you can change to another column like 'race/ethnicity')
sensitive_feature = 'gender'

# Define features and target
X = df.drop(columns=['math score', 'passed_math'])
y = df['passed_math']


# Step 3: Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)


# Step 4: Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


# Step 5: Train model
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)


# Step 6: Model performance
acc = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {acc:.3f}")


# Step 7: Fairlearn MetricFrame
mf = MetricFrame(
    metrics={
        'accuracy': accuracy_score,
        'selection_rate': selection_rate,
        'true_positive_rate': true_positive_rate,
    },
    y_true=y_test,
    y_pred=y_pred,
    sensitive_features=df.loc[y_test.index, sensitive_feature]
)

print("\n=== Fairness Metrics by Sensitive Group ===")
print(mf.by_group)


# Step 8: Equal Opportunity Difference
eo_diff = mf.difference(method='between_groups')['true_positive_rate']
print(f"\nEqual Opportunity Difference: {eo_diff:.3f}")


# Step 9: Disparate Impact
# Ratio of minimum to maximum selection rate across groups
sr = mf.by_group['selection_rate']
disparate_impact = sr.min() / sr.max()
print(f"Disparate Impact (min/max selection rate): {disparate_impact:.3f}")


# Step 10: Predictive Parity
# Precision parity = ratio or difference of precision between groups
precision = {}
for g in df[sensitive_feature].unique():
    y_true_g = y_test[df.loc[y_test.index, sensitive_feature] == g]
    y_pred_g = y_pred[df.loc[y_test.index, sensitive_feature] == g]
    tn, fp, fn, tp = confusion_matrix(y_true_g, y_pred_g).ravel()
    precision[g] = tp / (tp + fp + 1e-9)
print(f"\nPredictive Parity by Group: {precision}")


# Step 11: Threshold Limit Analysis
print("\n=== Threshold Fairness Check ===")
y_prob = model.predict_proba(X_test)[:, 1]
thresholds = np.arange(0.1, 1.0, 0.1)
for t in thresholds:
    y_pred_t = (y_prob >= t).astype(int)
    tpr = true_positive_rate(y_test, y_pred_t)
    print(f"Threshold {t:.1f}: True Positive Rate = {tpr:.3f}")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283834 sha256=a6862b558d128737501e46117b2e9de8f6b929e8d9e2d2ddc51071e76fdf1320
  Stored in directory: /root/.cache/pip/wheels/e7/5d/0e/4b4fff9a47468fed5633211fb3b76d1db43fe806a17fb7486a
Successfully built lime
Model Accuracy: 0.893

=== Fairness Metrics by Sensitive Group ===
        accuracy  selection_rate  true_positive_rate
gender                                              
0       0.902098        0.832168            0.956522
1       0.885350        0.891720            0.955224

Equal Opportunity Difference: 0.001
Disparate Impact (min/max selection rate): 0.933

Predictive Parity by Group: {np.int64(0): np.float64(0.9243697478913918), np.int64(1): np.float64(0.9142857142791837)}

=== Threshold Fairness Check ===
Threshold 0.1: True Positive Rate = 1.000
Threshold 0.2: True Positive Rate = 0